# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.core import Model
from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.widgets import RunDetails
import json

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
# Load workspace.
ws = Workspace.from_config()

# Choose a name for experiment and create a new experiment.
experiment_name = 'udacity-exp-hyp-d'
experiment=Experiment(ws, experiment_name)

# Load the registered dataset from the workspace.
dataset = Dataset.get_by_name(ws, name='heart_failure_data')

# Display the head of the dataset.
dataset.to_pandas_dataframe().head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [3]:
# Create compute cluster. If it does not exist then create one.
cpu_cluster_name = "udacity-comp-cst"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cluster")
    
except ComputeTargetException:    
    compute_config = AmlCompute.provisioning_configuration(vm_size = "Standard_DS12_V2", 
                                                           max_nodes=4,
                                                          idle_seconds_before_scaledown=600,
                                                          vm_priority='lowpriority')
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [21]:
# List curated environments for reference.
envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("AzureML"):
        print("Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())

Name AzureML-AutoML-GPU
packages channels:
- anaconda
- conda-forge
- pytorch
dependencies:
- python=3.6.2
- pip=20.2.4
- pip:
  - azureml-core==1.19.0
  - azureml-pipeline-core==1.19.0
  - azureml-telemetry==1.19.0
  - azureml-defaults==1.19.0
  - azureml-interpret==1.19.0
  - azureml-automl-core==1.19.0
  - azureml-automl-runtime==1.19.0
  - azureml-train-automl-client==1.19.0
  - azureml-train-automl-runtime==1.19.0
  - azureml-dataset-runtime==1.19.0
  - inference-schema
  - py-cpuinfo==5.0.0
  - boto3==1.15.18
  - botocore==1.18.18
- numpy~=1.18.0
- scikit-learn==0.22.1
- pandas~=0.25.0
- fbprophet==0.5
- holidays==0.9.11
- setuptools-git
- psutil>5.0.0,<6.0.0
name: azureml_21a55bb777255b4a59b1462d50a96839

Name AzureML-AutoML-DNN-GPU
packages channels:
- anaconda
- conda-forge
- pytorch
dependencies:
- python=3.6.2
- pip=20.2.4
- pip:
  - werkzeug==0.16.1
  - azureml-core==1.19.0
  - azureml-pipeline-core==1.19.0
  - azureml-telemetry==1.19.0
  - azureml-defaults==1.19.0
  - azur

In [4]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({"--C": uniform(1,5),
                                        "--max_iter": choice(10,50,100) })

#TODO: Create your estimator and hyperdrive config

# Create an environment for the training.
env = Environment(name="udacity-env")

# Install packages to the environment.
for pip_package in ["scikit-learn","pandas","numpy","azureml-core","azureml-defaults"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      arguments=['--data', dataset.id],
                      compute_target=cpu_cluster,
                      environment=env)          

hyperdrive_run_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=20,
                             max_concurrent_runs=4)

In [5]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [33]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [34]:
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

#Get list of associated filesnames.
print(best_run.get_file_names())

['--data', '40ef3347-dead-4a72-89e0-0beba1022f30', '--C', '2.4939183275009276', '--max_iter', '100']
['azureml-logs/55_azureml-execution-tvmps_6993aca32be3b622712d0b39a33a685f7d6397327301439d590cc787d7850bc7_p.txt', 'azureml-logs/65_job_prep-tvmps_6993aca32be3b622712d0b39a33a685f7d6397327301439d590cc787d7850bc7_p.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_6993aca32be3b622712d0b39a33a685f7d6397327301439d590cc787d7850bc7_p.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/107_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_l_7dfcdc21-dca5-4ae6-86bc-542506258a80.jsonl', 'logs/azureml/dataprep/python_span_l_7dfcdc21-dca5-4ae6-86bc-542506258a80.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']


In [54]:
# Return best run properties.
print(best_run.get_details()['properties'])

{'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '091fd7a0-3b7d-442e-950a-cacea455a23c', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}


In [35]:
#TODO: Save the best model

# Download the best run model file.
best_run.download_file('outputs/model.joblib','outputs/mymodel.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [43]:
# Register best model to Azure ML
model = best_run.register_model(model_name='hyperdrive-best',model_path='outputs/model.joblib')

In [46]:
# Set the name of the endpoint service you will deploy. 
service_name = 'hyperdrive-best-sv'

# Define the inference and deployment config.
inference_config = InferenceConfig(entry_script='score_hd.py', environment=env)
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

# Deploy the model on Azure.
service = Model.deploy(workspace = ws,
                       name = service_name,
                       models = [model],
                       inference_config = inference_config,
                       deployment_config = deployment_config)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..........................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [47]:
# Enable application insights for the logging
service.update(enable_app_insights=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [48]:
test_data = [{"age": 75, 
 "anaemia": 0, 
 "creatinine_phosphokinase": 582, 
 "diabetes": 0, 
 "ejection_fraction": 20, 
 "high_blood_pressure": 1, 
 "platelets": 265000, 
 "serum_creatinine": 1.9, 
 "serum_sodium": 130, 
 "sex": 1, 
 "smoking": 0, 
 "time": 4},

{"age": 75, 
 "anaemia": 0, 
 "creatinine_phosphokinase": 582, 
 "diabetes": 0, 
 "ejection_fraction": 20, 
 "high_blood_pressure": 1, 
 "platelets": 265000, 
 "serum_creatinine": 1.9, 
 "serum_sodium": 130, 
 "sex": 1, 
 "smoking": 0, 
 "time": 4}]

print(test_data)

[{'age': 75, 'anaemia': 0, 'creatinine_phosphokinase': 582, 'diabetes': 0, 'ejection_fraction': 20, 'high_blood_pressure': 1, 'platelets': 265000, 'serum_creatinine': 1.9, 'serum_sodium': 130, 'sex': 1, 'smoking': 0, 'time': 4}, {'age': 75, 'anaemia': 0, 'creatinine_phosphokinase': 582, 'diabetes': 0, 'ejection_fraction': 20, 'high_blood_pressure': 1, 'platelets': 265000, 'serum_creatinine': 1.9, 'serum_sodium': 130, 'sex': 1, 'smoking': 0, 'time': 4}]


In [51]:
# Create a JSON payload and then pass it to the new endpoint for inference.
input_payload = json.dumps({
    'data': test_data
})

# print(input_payload)

output = service.run(input_payload)
print(output)

[1, 1]


TODO: In the cell below, print the logs of the web service and delete the service

In [52]:
# Return the service logs for the new endpoint.
logs = service.get_logs()
print(logs)

2021-01-11T09:38:42,937626035+00:00 - rsyslog/run 
2021-01-11T09:38:42,944216432+00:00 - nginx/run 
2021-01-11T09:38:42,945629053+00:00 - iot-server/run 
/usr/sbin/nginx: /azureml-envs/azureml_be0a3b2a96e26af447d0cf8cd6a8b7b3/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_be0a3b2a96e26af447d0cf8cd6a8b7b3/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_be0a3b2a96e26af447d0cf8cd6a8b7b3/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_be0a3b2a96e26af447d0cf8cd6a8b7b3/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_be0a3b2a96e26af447d0cf8cd6a8b7b3/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-01-11T09:38:42,961993393+00:00 - gunicorn/run 
EdgeHubC

In [ ]:
# Delete the service.
service.delete()